In [ ]:
%load_ext google.cloud.bigquery

In [ ]:
%%bigquery
-- task 1
SELECT 
    SUM(cumulative_confirmed) as total_cases_worldwide
  FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
  WHERE 
    date= '2020-04-10'

In [ ]:
%%bigquery
-- task 2
select count(*) as count_of_states
from (
SELECT 
  subregion1_name as state, 
  sum(cumulative_deceased) as death_count
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
where country_name='United States of America' and date='2020-04-10' and subregion1_name is NOT NULL
group by subregion1_name
)
where death_count > 300

In [ ]:
%%bigquery
-- task 3
SELECT 
    subregion1_name AS state,
    SUM(cumulative_confirmed) AS total_confirmed_cases
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE country_name="United States of America" AND date='2020-04-10' AND subregion1_name IS NOT NULL
GROUP BY subregion1_name
HAVING total_confirmed_cases > 1000
ORDER BY total_confirmed_cases DESC

In [ ]:
%%bigquery
-- Task 4. Fatality ratio
SELECT 
    SUM(cumulative_confirmed) AS total_confirmed_cases,
    SUM(cumulative_deceased) AS total_deaths,
    (SUM(cumulative_deceased)/SUM(cumulative_confirmed))*100 AS case_fatality_ratio
    
  FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
  WHERE 
    country_name='Italy' 
    AND date BETWEEN '2020-06-01' AND '2020-06-30'


In [ ]:
%%bigquery
-- Task 5. Identifying specific day
SELECT 
    date    
  FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
  WHERE 
    country_name='Italy'
    AND cumulative_deceased > 12000
  GROUP BY date
  ORDER BY date
  LIMIT 1



In [ ]:
%%bigquery
-- Task 6. Finding days with zero net new cases
WITH india_cases_by_date AS (
  SELECT
    date,
    SUM(cumulative_confirmed) AS cases
  FROM
    `bigquery-public-data.covid19_open_data.covid19_open_data`
  WHERE
    country_name="India"
    AND date between '2020-02-21' and '2020-03-12'
  GROUP BY
    date
  ORDER BY
    date ASC
 )
, india_previous_day_comparison AS
(SELECT
  date,
  cases,
  LAG(cases) OVER(ORDER BY date) AS previous_day,
  cases - LAG(cases) OVER(ORDER BY date) AS net_new_cases
FROM india_cases_by_date
)
SELECT COUNT(*)
FROM india_previous_day_comparison
WHERE net_new_cases = 0

In [ ]:
%%bigquery
-- Task 7. Doubling rate
WITH USA_cases_by_date AS (
  SELECT
    date,
    SUM(cumulative_confirmed) AS cases
  FROM
    `bigquery-public-data.covid19_open_data.covid19_open_data`
  WHERE
    country_name="United States of America"
    AND date between '2020-03-22' and '2020-04-20'
  GROUP BY
    date
  ORDER BY
    date ASC
 )
, USA_previous_day_comparison AS
(SELECT
    Date,
    cases as Confirmed_Cases_On_Day,
    LAG(cases) OVER(ORDER BY date) AS Confirmed_Cases_Previous_Day ,
    (cases - LAG(cases) OVER(ORDER BY date))*100/LAG(cases) OVER(ORDER BY date) AS Percentage_Increase_In_Cases
    
  FROM USA_cases_by_date
)
SELECT 
  Date, Confirmed_Cases_On_Day, Confirmed_Cases_Previous_Day, Percentage_Increase_In_Cases
FROM USA_previous_day_comparison
WHERE Percentage_Increase_In_Cases > 5



In [ ]:
%%bigquery
-- Task 8. Recovery rate
SELECT country, recovered_cases, confirmed_cases, recovery_rate
FROM (
  SELECT
    country_name country,
    SUM(cumulative_recovered) recovered_cases,
    SUM(cumulative_confirmed) confirmed_cases,
    (SUM(cumulative_recovered)/SUM(cumulative_confirmed))*100 recovery_rate
    FROM
      `bigquery-public-data.covid19_open_data.covid19_open_data`
  WHERE date = '2020-05-10'
  GROUP BY country
  )
WHERE confirmed_cases > 50000
ORDER BY recovery_rate DESC
LIMIT 5


In [ ]:
%%bigquery
-- Task 9. CDGR - Cumulative daily growth rate
WITH
  france_cases AS (
  SELECT
    date,
    SUM(cumulative_confirmed) AS total_cases
  FROM
    `bigquery-public-data.covid19_open_data.covid19_open_data`
  WHERE
    country_name="France"
    AND date IN ('2020-01-24',
      '2020-05-10')
  GROUP BY
    date
  ORDER BY
    date)
, summary as (
 SELECT
  total_cases AS first_day_cases,
  LEAD(total_cases) OVER(ORDER BY date) AS last_day_cases,
  DATE_DIFF(LEAD(date) OVER(ORDER BY date),date, day) AS days_diff
FROM
  france_cases
LIMIT 1
)
SELECT 
    first_day_cases, 
    last_day_cases, 
    days_diff, 
    POW(last_day_cases/first_day_cases,1/days_diff)-1 as cdgr
FROM summary


In [ ]:
%%bigquery
-- Task 10
SELECT
    date, SUM(cumulative_confirmed) AS country_cases,
    SUM(cumulative_deceased) AS country_deaths
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE date BETWEEN '2020-03-15' AND '2020-04-30' AND country_name ="United States of America"
GROUP BY date
-- Click "explore in looker studio"